In [1]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import time
import imutils
from imutils.video import VideoStream
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

In [2]:
# 사진 속에서 얼굴을 탐지하는 face_detector 모델
faceNet = cv2.dnn.readNet('models/face_detector/deploy.prototxt','models/face_detector/res10_300x300_ssd_iter_140000.caffemodel')
# 얼굴인식 후 마스크 착용 여부를 확인하는 모델
maskNet = load_model('models/mask_detector.model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:
def drawPred(frame, conf, locs, preds):
    for loc in locs:
        cv2.rectangle(frame, (loc[0], loc[1]), (loc[2], loc[3]), (0, 0, 255), 2)

        label = '%.2f' % conf

        cv2.putText(frame, label, (loc[0], loc[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255))
    return frame


In [4]:
def detect_face(frame, h, w, faceNet): 
    blob = cv2.dnn.blobFromImage(frame, scalefactor=1.0, size=(300, 300), mean=(104.0,177.0,123.0))
    faceNet.setInput(blob)
    detections = faceNet.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0,0,i,2]

        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            face = frame[startY:endY, startX:endX]
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            faces.append(face)
            locs.append((startX, startY, endX, endY))
        
    return np.array(faces, dtype="float32"), locs

In [5]:
def predict_mask(faces, maskNet):
    return maskNet.predict(faces, batch_size=32)

In [15]:
def draw_bounding_box(locs, preds):
    for(box, pred) in zip(locs, preds):
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred
        
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        cv2.putText(frame, label, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

In [7]:
def draw_human_bounding_box(locs, frame):
    for box in locs:
        (startX, startY, endX, endY) = box
        cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
    return frame

In [8]:
def printResolution():
    vs = VideoStream(src=0).start()
    frame = vs.read()
    print(frame.shape)

printResolution()

(1080, 1920, 3)


## Third-Party Camera

In [16]:
vs = cv2.VideoCapture(0)
time.sleep(2.0)

while True:
    ret, frame = vs.read()
    frame = imutils.resize(frame, width=1400)
    h, w = frame.shape[:2]
    
    faces = []
    locs = []
    preds = []
    
    (faces, locs) = detect_face(frame, h, w, faceNet)
    
    if len(faces) > 0:
        preds = predict_mask(faces, maskNet)
        
    draw_bounding_box(locs, preds)

    cv2.imshow("Frame", frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

# vs.stop()
cv2.destroyAllWindows()	

AttributeError: 'NoneType' object has no attribute 'shape'

### FaceTime HD Camera

In [ ]:
vs = cv2.VideoCapture(1)
time.sleep(2.0)

while True:
    ret, frame = vs.read()
    frame = imutils.resize(frame, width=1400)
    h, w = frame.shape[:2]
    
    faces = []
    locs = []
    preds = []
    
    (faces, locs) = detect_face(frame, h, w, faceNet)
    
    if len(faces) > 0:
        preds = predict_mask(faces, maskNet)
        
    draw_bounding_box(locs, preds)

    cv2.imshow("Frame", frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

# vs.stop()
cv2.destroyAllWindows()	